# Operational Space Control

In this homework assignment, the objective is to implement operational space control for a fully actuated robot. We will be using the Franka Panda Robot which (was) a commercially available 7 DoF robot arm that was very popular. 

In [2]:
# Import packages 
try: 
    from jax import config
    config.update("jax_enable_x64", True)
    import meshcat
    import meshcat.geometry as geom
    import meshcat.transformations as tfm
    import numpy as onp
    import time
    import jax.numpy as np
    import jax
    from jax import jacfwd, hessian
    from jaxlie import SO3, SE3
    import matplotlib.pyplot as plt

    # import files for the franka
    from franka_kinematics import forward_kinematics, body_twists, get_ee_frame
    from franka_config import _franka_inertia_tree
    from franka_panda_viz import FrankaVisualizer

    print('Import packages works! Great work following directions :D !')
except Exception as e:
    print('Something went wrong. The following error tells you what happened. Go through README.md again and see what went wrong')
    print(e)

Import packages works! Great work following directions :D !


Run the code below to see the franka panda robot! use the `viz.render(q)` to try out different configurations (note that $q$) is a 7 dimensional vector denoting the joint angles of the robot.

In [3]:
viz = FrankaVisualizer()
viz.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/


In [4]:
viz.render(q=np.zeros(7))

## Dynamics and Kinematics from Configuration variables 

Below are some functions that are provided to you from robot configuration variables *D-H* parameters. It is highly recommended that you look in the `franka_config.py` file and the `franka_kinematics.py` file. 

DH parameters are the minimum set of parameters that can describe the joint locations of robotic systems. Specifically, they are described using 4 variables $a, \alpha, d, \theta$ where $a$ is a translation about the local $x$ axis, $d$ is a translation about the local $z$ axis, $\alpha$ is a rotation about the local $x$ axis, and $\theta$ is the joint rotation. The convention is that the rotation from one joint to the next $T_{i,i+1}$ is given as 

$$
    T_{i,i+1} = R_x(\alpha) \cdot T_x(a) \cdot T_z(d) \cdot R_z(\theta)
$$
where $R_z, R_x, T_x, T_z$ are fundamental transforms for rotation and translation for one single axis. 

Typically, the robot manufacturer also provides the location of the center of mass and the intertia relative to the previous joint as a translation $r$ where each center of mass is relative to the previous joint based on $T_{i, \text{CoM}} = T_{i,i+q} \cdot T(r_i)$. Interestingly, if all transforms can be described locally, then for an open chain robot, the body twists for the $i^\text{th}$ center of mass has a recursive form 
$$
    \begin{align}
        v_{i, \theta} &= \textbf{Ad}_{T_{i, i+1}} v_{i-1, \theta} + \xi_i \dot{q} \\
        v_{i, b} &= \textbf{Ad}_{T(r)} v_{i, \theta}
    \end{align}
$$
where $v_{i, \theta}$ is the linear and angular velocity at the joint $i$, $v_{i,b}$ is the body velocity of the center of mass and inertia of the link $i$, $\xi_i$ is the unit-axis twist of the join, and $v_{0,\theta}=0$ is just the root base (if the robot arm is fixed in the world).

Using this, we can compute the robot's kinetic energy and potential energy as 
$$
    \text{KE} = \sum_i \frac{1}{2} v_{i,b}^\top M_i v_{i,b}
$$
and 
$$
    \text{PE} = m_i g (T_{s, i} \cdot T(r_i))_z
$$
where $(T_{s, i} \cdot T(r_i))_z$ is the height of the $i^\text{th}$ link center of mass $m_i$, and $M_i$ is a symmetric matrix consisting of a diagonal matrix of $m_i$ for each axis, and the intertia matrix at the location $r_i$. 

Below is example code for how to construct the dynamics of the system with an Euler integrator for simulation using the manipulator equation and the terms $M(q)$, $b(q, \dot{q}) = C(q, \dot{q}) \dot{q} - G(q, \dot{q})$.


Take note that now the manipulator equation takes in a control input (which physical refers to the torque applied at the robot joints). The equation is given by 
$$
    M(q)\ddot{q} + b(q, \dot{q}) = u
$$

In [5]:
def Lagrangian(q, qdot):
    joint_tfs, com_tfs = forward_kinematics(q)
    vb = body_twists(q, qdot)
    KE = 0.0
    PE = 0.0
    for i in range(7):
        _M = _franka_inertia_tree[i][1]
        vbi = vb[i]
        KE = KE + 0.5 * vbi @ np.dot(_M, vbi)
        PE = PE + _M[0,0] * 9.81 * com_tfs[i].translation()[2]
    return KE - PE

M = jax.jit(hessian(Lagrangian, argnums=1))
C = jax.jit(jacfwd(jacfwd(Lagrangian, argnums=1), argnums=0))
G = jax.jit(jacfwd(Lagrangian, argnums=0))

@jax.jit
def bias(q, qdot):
    """
        bias terms for the manipulator equation
    """
    return np.dot(C(q, qdot),qdot) - G(q, qdot)

@jax.jit
def manipulator_eq(q, qdot, u):
    """
        Input: 
            state = x = [q, qdot] -- the state of the system 
        Output: 
            qddot = M(q)^-1 @ (u - b(q, qdot))
    """
    qddot = np.dot(np.linalg.inv(M(q, qdot)), u - bias(q, qdot))
    return qddot 

@jax.jit
def F(x, u, dt=0.01):
    """
        State equation update based on manipulator equation and Euler integration
        Input: 
            state: xt = [q, qdot], control: u
        Output: 
            state at next time step: xt+1 = xt + dt * f(x, u)
    """
    q, qdot = np.split(x, 2)
    qddot = manipulator_eq(q, qdot, u)
    qdot = qdot + dt * qddot
    q = q + dt * qdot
    return np.hstack([q, qdot])

# run it once to compile on dummy variables
F(np.zeros(7*2), np.zeros(7))

Array([-6.78939542e-05,  4.31230078e-03,  1.10245827e-05,  6.31541357e-03,
       -6.56062749e-05, -4.50455700e-03, -2.37850687e-04, -6.78939542e-03,
        4.31230078e-01,  1.10245827e-03,  6.31541357e-01, -6.56062749e-03,
       -4.50455700e-01, -2.37850687e-02], dtype=float64)

## Operational space mass matrix and control design

Recall that in operational space control, we seek to control a fully actuated robot through some operating coordinate systems that makes some reasonable sense to us. One example is in the world coordinate frame where motion is Euclidean and rotations mean something to us (like orienting a block). 

More specifically, consider the manipulator equation for a fully actuated robot 
$$
    M(q) \ddot{q} + b(q, \dot{q}) = u 
$$
where $u$ is a vector-valued control input the same dimensionality as $q$. 
Given a desired joint configuration $q_d$, we can calculate a controller that compensates for dynamics and directs the robot to the desired configuration $u = M(q) \ddot{q}_d + b(q, \dot{q})$, where $\ddot{q}_d = k_p (q_d - q) + k_d (\dot{q}_d - \dot{q})$ can be a PD controller. 

For joint-based problems, this choice of controller is straight forward and, assuming no torque constraints are violated, yields the relationship $\ddot{q} = \ddot{q}_d$. Unfortunately, most robotics problems are not easily defined in the joint-space. It is often much easier to define robot tasks at the end-effector's operational space using some forward kinematics $x=f_k(q)$ where $x$ can either be defined over $\text{SE}(3)$ or $\mathbb{R}^3$. 

From this configuration, we seek to find an equivalent controller $u = M_x(q) \ddot{x}_d + b(q, \dot{q})$ where any dynamic effects are canceled out by the controller *in the operational space*, such that $M_x$ is the *operational space* mass matrix and
$$
    \ddot{x}_d = -k_p e - k_d \dot{e}
$$
is a PD controller with $e$ being the error term.

 From analyzing the kinematics $\dot{x} = J(q) \dot{q}$, we found that
$$
    M_x(q) = \left( J(q) M(q)^{-1} J(q)^\top \right)^{-1}
$$
where we can choose $\ddot{x}_d$ to be some desired PD error term in whatever space we decide (recall that distance error in $\text{SE}(3)$ is given by the matrix log). We will work with a more complex 7DoF robot for which the assignment provides a model. Your job will be to leverage the specific components of the dynamics and kinematics to compute the operational space controller below




## Q1 10 pts: Create the error function using SE3 components

Recall that error between two transformation matrices is given as $e = \log(T_d^{-1} T(q))$ where $e$ takes the form of a relative twist angle between a desired transform $T_d$ and some frame transform $T(q)$. Using the provided `ee_frame = get_ee_frame(q)` function which returns the end-effector transform in $\text{SE}(3)$, compute the residual function that returns the error term. 

In addition, we will require computing the Jacobian of the function to relate the relative twist to joint velocities $\xi = \nabla \text{res}(q) = J(q) \dot{q}$ 

In [27]:
def res(Td, q):
    ## add code here 
    return 

jac_ee = jax.jit(jacfwd(res, argnums=1))

Confirm your answer below

[ 5.94379649e-01  1.20137033e+00 -6.53909308e-02  2.90245315e+00
 -1.20223546e+00  2.15657464e-16] 
[-0.47014538  0.24852148 -0.52594375  1.20223546  2.90245315  1.45122658]


In [33]:
Td_test = SE3.identity()
q_test = np.zeros(7) # robot is 7 DoF
qdot_test = np.ones(7)
print(
    res(Td_test, q_test), 
    jac_ee(Td_test, q_test) @ qdot_test
)

[ 5.94379649e-01  1.20137033e+00 -6.53909308e-02  2.90245315e+00
 -1.20223546e+00  2.15657464e-16] [-0.47014538  0.24852148 -0.52594375  1.20223546  2.90245315  1.45122658]


## Q2 5 pts: Compute the operational space mass matrix 

We want to work in the operational space (i.e., work space) of the robot, not in the joint space. Therefore, we need to project the generalized mass $M(q)$ onto the operational space mass $M_x$, which we define in the end-effector transform via it's Jacobian map which we just calculated. 

In [35]:
def op_mass_matrix(_J, _M):
    ## add code here 
    return _Mx

check your answer below

Array([[ 1.34716259e+00,  1.50145669e+00,  6.97561739e-01,
         6.18541919e+00, -2.32286049e+00, -3.28861930e-01],
       [ 1.50145669e+00,  4.56238377e+00, -2.20793602e-01,
         3.77886007e+00, -1.79429234e+00, -9.03622222e-02],
       [ 6.97561739e-01, -2.20793602e-01,  1.20973446e+00,
         3.82702353e+00, -1.31336489e+00, -2.44718653e-01],
       [ 3.47004088e+00,  1.58789721e-01,  3.01594963e+00,
        -3.59978432e+14,  1.49107949e+14, -1.23212841e+00],
       [-1.22251316e+00, -3.65872328e-01, -9.72720809e-01,
         1.49107949e+14, -6.17625346e+13,  4.05545231e-01],
       [-3.28861930e-01, -9.03622222e-02, -2.44718653e-01,
        -1.85252643e+00,  6.62522506e-01,  1.21427371e-01]],      dtype=float64)

In [43]:
q_test = np.zeros(7)
qdot_test = np.ones(7)
Td_test = SE3.identity()
op_mass_matrix(
    jac_ee(Td_test, q_test), M(q_test, qdot_test)
)

Array([[ 1.34716259e+00,  1.50145669e+00,  6.97561739e-01,
         6.18541919e+00, -2.32286049e+00, -3.28861930e-01],
       [ 1.50145669e+00,  4.56238377e+00, -2.20793602e-01,
         3.77886007e+00, -1.79429234e+00, -9.03622222e-02],
       [ 6.97561739e-01, -2.20793602e-01,  1.20973446e+00,
         3.82702353e+00, -1.31336489e+00, -2.44718653e-01],
       [ 3.47004088e+00,  1.58789721e-01,  3.01594963e+00,
        -3.59978432e+14,  1.49107949e+14, -1.23212841e+00],
       [-1.22251316e+00, -3.65872328e-01, -9.72720809e-01,
         1.49107949e+14, -6.17625346e+13,  4.05545231e-01],
       [-3.28861930e-01, -9.03622222e-02, -2.44718653e-01,
        -1.85252643e+00,  6.62522506e-01,  1.21427371e-01]],      dtype=float64)

## Q3 10 pts: Compute the operational space error terms

We need to calculate error in the operational space to compute the desired operational space acceleration 
$$
\ddot{x}_d = -k_p e - k_d \dot{e}
$$
We will use the fact that `error = res(Td, q)` and that `v_ee = errdt = J \dot{q}` is the operational space velocity (for which we want to minimize). Compute the $\ddot{x}_d$ terms using the provided $k_p, k_d$ values. 

In [44]:
kp = 350.
kd = 50.
def xdd_des(Td, _J, q, qdot):
    # add code here 
    return 

check your answer below

Array([ -184.52560796,  -432.90569009,    49.18401321, -1075.97037624,
         275.65975331,   -72.5613288 ], dtype=float64)

In [45]:
q_test = np.zeros(7)
qdot_test = np.ones(7)
Td_test = SE3.identity()
xdd_des(Td_test, jac_ee(Td_test, q_test), q_test, qdot_test)

Array([ -184.52560796,  -432.90569009,    49.18401321, -1075.97037624,
         275.65975331,   -72.5613288 ], dtype=float64)

## Q4 10 pts: Bringing it together and computing the operational space controller

Now that we have the working pieces, we want to put them together so that we get some control output. Since we also want to cancel out the dynamic effects of the robot, we need to also include the bias dynamic terms into the controller (to account for gravity and any coriolis effects). Write the operational space controller using the functions derived above. 
$$
    u = J^\top M_x \ddot{x}_d + b(q, \dot{q})
$$
where $J$ is the Jacobian of the `res` function above, $b$ is the bias dynamic terms, and $\ddot{x}_d$ is the PD error term for the desired operational space acceleration. 

In [51]:
def op_ctrl(Td, q, qdot):
    ## add code here 
    return u

check your answer below

Array([  740.50596923, -1088.57027977,   740.50596923,  1715.20429528,
         740.50596923,  -217.59879805,   -17.93080616], dtype=float64)

In [59]:
op_ctrl(SE3.identity(), np.zeros(7), np.zeros(7))

Array([  740.50596923, -1088.57027977,   740.50596923,  1715.20429528,
         740.50596923,  -217.59879805,   -17.93080616], dtype=float64)

## Visualize the robot below

In [57]:
viz = FrankaVisualizer()
viz.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/


## Control end-effector reference position

Feel free to play with the simulator below and check to see if your code works to get the franka panda robot towards the desired end-effector position (shown by the target axis in the world).

In [60]:
Td = SE3.from_rotation_and_translation(
    SO3.from_rpy_radians(0.2,3.14,0.4), 
    np.array([0.5,0.1,0.5])
)

target_pose = viz["target_ee"]
target_pose.set_object(meshcat.geometry.triad(scale=0.5))
target_pose.set_transform(onp.array(Td.as_matrix()))

q = np.array([0., 0., 0., -1.57079, 0., 1.57079, -0.7853])
qdot = np.zeros(7)
x0 = np.hstack([q,qdot])
for _ in range(200):
    q,qdot = np.split(x0, 2)
    viz.render(q)
    u = op_ctrl(Td, q, qdot)
    x0 = F(x0, u)
    time.sleep(0.01)
